In [1]:
import pandas as pd
from langdetect import detect
import glob



In [2]:
all_files = glob.glob("./annotations/*.tsv")

li = []

for filename in all_files:

    df = pd.read_table(filename, index_col=None, header=0, sep='\t')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [3]:
frame

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path
0,917791044158185473,917791044158185473_0,informative,1.0000,informative,0.6766,other_relevant_information,1.0000,other_relevant_information,0.6766,NaN,NaN,RT @Gizmodo: Wildfires raging through Northern...,http://pbs.twimg.com/media/DLyi_WYVYAApwNg.jpg,data_image/california_wildfires/10_10_2017/917...
1,917791130590183424,917791130590183424_0,informative,1.0000,informative,0.6667,infrastructure_and_utility_damage,1.0000,affected_individuals,0.6667,NaN,NaN,PHOTOS: Deadly wildfires rage in California ht...,http://pbs.twimg.com/media/DLymKm9UMAAu0qw.jpg,data_image/california_wildfires/10_10_2017/917...
2,917791291823591425,917791291823591425_0,informative,0.6813,informative,1.0000,other_relevant_information,0.6813,infrastructure_and_utility_damage,1.0000,severe_damage,1.0,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,http://pbs.twimg.com/media/DLudaaZV4AAjT7x.jpg,data_image/california_wildfires/10_10_2017/917...
3,917791291823591425,917791291823591425_1,informative,0.6813,not_informative,1.0000,other_relevant_information,0.6813,not_humanitarian,1.0000,NaN,NaN,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,http://pbs.twimg.com/media/DLudaZXUMAABAEZ.jpg,data_image/california_wildfires/10_10_2017/917...
4,917792092100988929,917792092100988929_0,informative,0.6727,informative,0.6612,other_relevant_information,0.6727,infrastructure_and_utility_damage,0.6612,severe_damage,1.0,RT @TIME: California's raging wildfires as you...,http://pbs.twimg.com/media/DLwNe-NXUAE0XCw.jpg,data_image/california_wildfires/10_10_2017/917...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18077,916105333872918528,916105333872918528_0,informative,1.0000,not_informative,1.0000,rescue_volunteering_or_donation_effort,1.0000,not_humanitarian,1.0000,NaN,NaN,Carlos santana donates $100k to mexico earthqu...,http://pbs.twimg.com/media/DLao8g8W4AAKoPV.jpg,data_image/mexico_earthquake/6_10_2017/9161053...
18078,916105374134042624,916105374134042624_0,informative,1.0000,not_informative,0.6412,rescue_volunteering_or_donation_effort,1.0000,not_humanitarian,0.6412,NaN,NaN,Seth troxler donates 10k to earthquake relief ...,http://pbs.twimg.com/media/DLao-2GWsAA97vC.jpg,data_image/mexico_earthquake/6_10_2017/9161053...
18079,916112796194021376,916112796194021376_0,informative,1.0000,informative,0.6337,rescue_volunteering_or_donation_effort,1.0000,other_relevant_information,0.6337,NaN,NaN,Entercom/San Francisco Stations Raise Funds Fo...,http://pbs.twimg.com/media/DLavu1YV4AAgSJ_.jpg,data_image/mexico_earthquake/6_10_2017/9161127...
18080,916164076484653056,916164076484653056_0,informative,0.6580,informative,0.6374,other_relevant_information,0.6580,rescue_volunteering_or_donation_effort,0.6374,NaN,NaN,Mexico Earthquakes | International Medical Cor...,http://pbs.twimg.com/media/DLbeXxlWsAEl9xu.jpg,data_image/mexico_earthquake/6_10_2017/9161640...


In [4]:
frame.columns

Index(['tweet_id', 'image_id', 'text_info', 'text_info_conf', 'image_info',
       'image_info_conf', 'text_human', 'text_human_conf', 'image_human',
       'image_human_conf', 'image_damage', 'image_damage_conf', 'tweet_text',
       'image_url', 'image_path'],
      dtype='object')

In [5]:
frame = frame.drop_duplicates(subset='tweet_id', keep="first")

In [6]:
len(frame)

16058

In [7]:
frame.drop(frame.columns.difference(['text_info','tweet_text']), 1, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
frame.columns=["Informativeness","text"]

In [10]:
frame.Informativeness.value_counts()

informative        11503
not_informative     4555
Name: Informativeness, dtype: int64

In [11]:
### Keep only english tweets

def is_english(text):

    try:
          language = detect(text)
    except:
          #These will be removed
          print("This row throws and error:", text)
          return False;
    if(language=='en'):
      return True;
    else:
      return False;

frame_en=frame[frame.apply(lambda x: is_english(x['text']), axis=1)]

In [12]:
len(frame_en)

15738

In [14]:
frame_en.Informativeness.value_counts()

informative        11357
not_informative     4381
Name: Informativeness, dtype: int64

In [15]:
frame_en.head()

,Informativeness,text
0,informative,RT @Gizmodo: Wildfires raging through Northern...
1,informative,PHOTOS: Deadly wildfires rage in California ht...
2,informative,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...
4,informative,RT @TIME: California's raging wildfires as you...
5,informative,Wildfires Threaten Californiaâ€™s First Legal ...


In [16]:
frame_en['Informativeness']=frame_en['Informativeness'].map(lambda x: 1 if(x=='informative') else 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
frame_en["Informativeness"].value_counts()

1    11357
0     4381
Name: Informativeness, dtype: int64

In [18]:
frame_en.to_pickle("./../dataset2Cleaned.pkl")